In [2]:
import gradio as gr
import pandas as pd
import gensim
import pickle

from absa_functions import *

In [9]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\echeng\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\echeng\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [3]:
def greet(name):
  return "Hello " + name + "!!"

iface = gr.Interface(fn=greet, inputs="text", outputs="text")
iface.launch()

Running locally at: http://127.0.0.1:7860/
To create a public link, set `share=True` in `launch()`.
Interface loading below...


(<Flask 'gradio.networking'>, 'http://127.0.0.1:7860/', None)

In [4]:
corpus = pickle.load(open('corpus.pkl', 'rb'))
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')

lda_model = gensim.models.ldamodel.LdaModel.load('best_lda_model.gensim')

In [5]:
#numerical mapping for topics in LDA model
topic_map = {0: 'menu', 1: 'service', 2: 'miscellaneous', 3: 'place', 4: 'price', 5: 'food', 6: 'staff'}

In [10]:
raw_review = "The food was good, but it's not worth the wait--or the lousy service."
clean_sample = prepare_text_for_lda(raw_review)
sample_2bow = dictionary.doc2bow(clean_sample)
sample_2bow

[(5, 1), (31, 1), (114, 1), (144, 1), (1296, 1), (4032, 1)]

In [12]:
def lda_prediction(restaurant_review):
    lda_model = gensim.models.ldamodel.LdaModel.load('best_lda_model.gensim')
    topic_map = {0: 'menu', 1: 'service', 2: 'miscellaneous', 3: 'place', 4: 'price', 5: 'food', 6: 'staff'}

    clean_sample = prepare_text_for_lda(restaurant_review)
    sample_2bow = dictionary.doc2bow(clean_sample)

    topics = lda_model.get_document_topics(sample_2bow)
    topic_dict = {topic_map[x[0]]:x[-1] for x in topics}
    top_topics = sorted(topic_dict.items(), key = itemgetter(1), reverse = True)
    top_topics = {tup[0]: round(tup[1], 4) for tup in top_topics if tup[1] > 0.1}

    topic_df = pd.DataFrame(top_topics, index=[0]).transpose().reset_index()
    topic_df.columns = ['Topic', 'Confidence']
    return topic_df

In [13]:
lda_prediction(raw_review)

,Topic,Confidence
0,service,0.5051
1,place,0.3841


In [14]:
iface = gr.Interface(fn=lda_prediction, 
                    inputs=gr.inputs.Textbox(lines=2, placeholder='Enter restaurant review here...'),
                    outputs=gr.outputs.Dataframe(headers=['Topic', 'Confidence']),
                    examples=[
                        ['The restaurant is too dark, and the bathroom was not clean. Also, everyone there is rude.'],
                        ['Fabulous dinner & environment but the older waiters have a real sense of entitlement.'],
                        ['Entrees were way too expensive.'],
                    ])
iface.launch(share=True)

Running locally at: http://127.0.0.1:7861/
This share link will expire in 72 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted
Running on External URL: https://35844.gradio.app
Interface loading below...


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7861/',
 'https://35844.gradio.app')